In [ ]:
# IMPORT PACKAGES 
from PIL import Image, ImageDraw
import numpy as np
from os import path, makedirs
import time
from tifffile import imsave

%matplotlib inline
import matplotlib.path as pltpath
from matplotlib.pyplot import imshow

In [ ]:
# SUB-ROUTINES 

def image_build(J,X,Z,params):
    r = params['res']
    s = params['s']
    h = params['boundary_height']
    w = params['boundary_width']
    
    # configure data for image construction
    Zpx = (Z[0,:]+(w))*((s[0]/w/2)*r)
    Zpy = (Z[1,:]+(h))*((s[1]/h/2)*r)
    Zmx = (Z[3,:]+(w))*((s[0]/w/2)*r)
    Zmy = (Z[4,:]+(h))*((s[1]/h/2)*r)
    
    Xx1 = (X[0,:]+(w))*((s[0]/w/2)*r)
    Xy1 = (X[1,:]+(h))*((s[1]/h/2)*r)
    Xx2 = (X[0,J[1,:]]+(w))*((s[0]/w/2)*r)
    Xy2 = (X[1,J[1,:]]+(h))*((s[1]/h/2)*r)
    
    # define length of variables for iteration
    N = params['filaments']
    M = params['motorsX']

    # create new PIL image object
    im = Image.new('RGB', s)
    draw = ImageDraw.Draw(im)

    # draw IgG domain
    ctrx = (params['igg_center'][0]+(w))*((s[0]/w/2)*r)
    ctry = (params['igg_center'][1]+(h))*((s[1]/h/2)*r)
    radx = (params['igg_radius']*((s[0]/w/2)*r))
    rady = (params['igg_radius']*((s[1]/h/2)*r))
    draw.ellipse([ctrx-radx,ctry-rady,ctrx+radx,ctry+rady],fill=(0,155,155))

    # plot filament positions
    igg_off = np.where(Z[5,:]==0)[0]
    [draw.line(((Zpx[i],Zpy[i]),(Zmx[i],Zmy[i])),fill=(255,0,0)) for i in igg_off]
    igg_on = np.where(Z[5,:]==1)[0]
    [draw.line(((Zpx[i],Zpy[i]),(Zmx[i],Zmy[i])),fill=(255,0,255)) for i in igg_on]
    
    # find all bundled pairs of motors
    motors = (np.where(J[1,:]!=-1)[0]) # change to !=-1 when moving into data already zero indexed
    
    # plot motor bundles
    [draw.line(((Xx1[i],Xy1[i]),(Xx2[i],Xy2[i])),fill=(0,255,0)) for i in motors]

    # plot motor positions
    [draw.point([(Xx1[i],Xy1[i])],fill=(255,255,0)) for i in range(M)]    

    return np.asarray(im);

def motor_boundary_check(x,y,params,boundary):
    if ~boundary.contains_point(np.array([x,y])):
        h = params['boundary_height'];
        w = params['boundary_width'];
        b = [np.less(y,-1*h),np.greater(y,h),np.less(x,-1*w),np.greater(x,w)];
        # b[0] = y position below bottom boundary
        # b[1] = y position above top boundary
        # b[2] = x position left of left boundary
        # b[3] = x position right of right boundary
        if b[0] == True and b[2] == False and b[3] == False:
            y = -1*h;
        elif b[1] == True and b[2] == False and b[3] == False:
            y = h;
        elif b[2] == True and b[0] == False and b[1] == False:
            x = -1*w;
        elif b[3] == True and b[0] == False and b[1] == False:
            x = w;
        elif b[0] == True and b[2] == True:
            x = -1*w;
            y = -1*h;
        elif b[0] == True and b[3] == True:
            x = w;
            y = -1*h;
        elif b[1] == True and b[2] == True:
            x = -1*w;
            y = h;
        elif b[1] == True and b[3] == True:
            x = w;
            y = h; 
            
    return x,y
def motor_state_forward(j,J,X,params,boundary):
    # check if motor ungergoes activation event
    if J[0,j] == 0:
        # define state transition
        if params['activationRate']*params['tstep'] > np.random.uniform():
            J[0,j] = 1;
        # define motor diffusion movement
        X[0,j] = X[0,j] + np.sqrt(2*params['tstep']*params['diffusionRate'])*np.random.randn() + (0.008 * np.random.randn() + 0.005);
        X[1,j] = X[1,j] + np.sqrt(2*params['tstep']*params['diffusionRate'])*np.random.randn();
        X[0,j],X[1,j] = motor_boundary_check(X[0,j],X[1,j],params,boundary);
        
    
    # check if motor undergoes bundling event
    elif J[0,j] == 1:
        # condition 1: look for motor candidates to bundle with
        candidate1 = np.where(J[0,:] == 1)[0];
        
        # condition 2: look for motors within motorBundleRadius
        candidate2 = np.where(np.sqrt((X[0,candidate1]-X[0,j])**2 + \
                              (X[1,candidate1]-X[1,j])**2)<params['motorBundleRadius'])[0];

        # group the candidate lists together (returns indices of candidates)
        bundle_candidates = [candidate1[i] for i in candidate2];

        # check the probability of a bundling event
        if (len(bundle_candidates)*params['tstep']*params['bundleRate']>np.random.uniform() 
            and len(bundle_candidates)>1):
            # uniformly randomly select a motor to bundle with                              
            J[1,j] = bundle_candidates[int(np.ceil(len(bundle_candidates)*np.random.uniform()))-1];
            while J[1,j]==j:
                J[1,j] = bundle_candidates[int(np.ceil(len(bundle_candidates)*np.random.uniform()))-1];
            J[0,j] = 2;
        else:
            # diffusion of motor occurs if no bundling event
            X[0,j] = X[0,j] + np.sqrt(2*params['tstep']*params['diffusionRate'])*np.random.randn() + (0.008 * np.random.randn() + 0.005);
            X[1,j] = X[1,j] + np.sqrt(2*params['tstep']*params['diffusionRate'])*np.random.randn();
            X[0,j],X[1,j] = motor_boundary_check(X[0,j],X[1,j],params,boundary);     
        
    return J[0,j],J[1,j],X

def find_filaments_2D(Z,x,y,params):
    # test cadidates
    d0 = x - Z[0,:];
    d1 = Z[3,:] - Z[0,:];
    d2 = y - Z[1,:];
    d3 = Z[4,:] - Z[1,:];

    d = (d0*d1 + d2*d3) / \
        (d1**2 + d3**2)
    T1 = (d>0) & (d<1)

    var = ((d0*d3)-(d2*d1))**2 / \
          (d1**2+d3**2)
    dpp = d0**2 + d2**2
    dmm = (x-Z[3,:])**2 + (y-Z[4,:])**2

    T4 = np.where((var < params['r']**2) & (T1 == True) | \
         ((dpp < params['r']**2) ^ (dmm < params['r']**2)) & (T1 == False))[0]
    
    return len(T4),T4
def update_motor_position(x1,y1,x2,y2,xm,ym):
    var = ((xm-x1)*(x2-x1) + (ym-y1)*(y2-y1)) / ((x2-x1)**2 + (y2-y1)**2);
    if var>1:
        var = 1;
    elif var<0:
        var = 0;
    # calculate new x and y positions for motor
    new_xm = x1 + var*(x2-x1);
    new_ym = y1 + var*(y2-y1);
    
    return new_xm,new_ym
def motor_attach_detach(lfil,rfil,lx,ly,rx,ry,Z,params,boundary):
    # toggle switches; disallows right motor from performing event if switched (left motor event occured)
    fil_switch = 0;    
    diffuse_event = 0;
    detach_event = 0;
    
    # check left motor events
    if lfil<0:
        # find number of filament binding cadidates and their indices for left motor
        l_num_fils,l_fil_indices = find_filaments_2D(Z,lx,ly,params);  
        if l_num_fils*params['tstep']*params['p1']>np.random.uniform():
            # select a random filament for left motor to bind to
            lfil = l_fil_indices[int(np.ceil(l_num_fils*np.random.uniform()))-1];
            # set new position of newly attached left motor
            lx,ly = update_motor_position(Z[0,lfil],Z[1,lfil],Z[3,lfil],Z[4,lfil],lx,ly);
            # switch fil_switch toggle
            fil_switch = 1;
        else:
            diffuse_event = 1;
            lx = rx;
            ly = ry;
    else:
        # check if both motors are attached to the same filament
        # to see if left motor switches binding to a different filament
        if lfil == rfil and np.random.uniform()>0.5:
            fil_switch = 1;
            # find number of filament binding cadidates and their indices for left motor
            l_num_fils,l_fil_indices = find_filaments_2D(Z,lx,ly,params);
            # probability of binding event
            if l_num_fils*params['tstep']*params['p1']>np.random.uniform():
                # select a random filament for left motor to bind to
                lfil = l_fil_indices[int(np.ceil(l_num_fils*np.random.uniform()))-1];
                # set new position of newly attached left motor
                lx,ly = update_motor_position(Z[0,lfil],Z[1,lfil],Z[3,lfil],Z[4,lfil],lx,ly);
        # check for detachment/unbinding event of left motor
        if np.random.uniform()>0.5:   
            detach_event = 1;
            # probability of left motor detachment from filament
            if params['p0']*params['tstep']>np.random.uniform():
                lfil = -1;
                lx = rx;
                ly = ry;
    
    # check right motor events
    if rfil<0: # right motor not bound to filament
        rx = lx;
        ry = ly;
        # find number of filament binding cadidates and their indices for right motor
        r_num_fils,r_fil_indices = find_filaments_2D(Z,rx,ry,params);
        # probability of binding event
        if r_num_fils*params['tstep']*params['p1']>np.random.uniform():
            # select a random filament for right motor to bind to
            rfil = r_fil_indices[int(np.ceil(r_num_fils*np.random.uniform()))-1];
            # set new position of newly attached right motor
            rx,ry = update_motor_position(Z[0,rfil],Z[1,rfil],Z[3,rfil],Z[4,rfil],rx,ry);
            if lfil<0:
                lx = rx;
                ly = ry;
        # check if both motors in bundle unbound to a filament; diffusion event occurs
        elif diffuse_event == 1:
            lx = lx + np.sqrt(2*params['tstep']*params['diffusionRate'])*np.random.randn();
            ly = ly + np.sqrt(2*params['tstep']*params['diffusionRate'])*np.random.randn();
            # check if new positions are within boundary
            lx,ly = motor_boundary_check(lx,ly,params,boundary);
            rx = lx;
            ry = ly;
    else: # right motor bound to filament
        # check if right motor and left motor are bound to same filament
        # and if left motor did not have a filament switch event
        if rfil==lfil and fil_switch == 0:
            # find number of filament binding cadidates and their indices for right motor
            r_num_fils,r_fil_indices = find_filaments_2D(Z,rx,ry,params);
            # probability of binding event
            if r_num_fils*params['tstep']*params['p1']>np.random.uniform():
                # select a random filament for right motor to bind to
                rfil = r_fil_indices[int(np.ceil(r_num_fils*np.random.uniform()))-1];
                # set new position of newly attached right motor
                rx,ry = update_motor_position(Z[0,rfil],Z[1,rfil],Z[3,rfil],Z[4,rfil],rx,ry);
        # check if both detachment event for left motor and filament switch of 
        # either motor did not occur
        if detach_event == 0 and fil_switch == 0:    
            # probability of right motor detachment from filament
            if params['p0']*params['tstep']>np.random.uniform():
                rfil = -1;
                rx = lx;
                ry = ly;
        
    return lfil,rfil,lx,ly,rx,ry

def bound_motor_movement(lfil,rfil,lx,ly,rx,ry,Z,params):
    # left motor not bound, right motor bound to filament; right motor walks
    if lfil==-1 and rfil!=-1:
        cb = np.sqrt((Z[3,rfil] - rx)**2 + (Z[4,rfil] - ry)**2);
        # motor travels past end of filament, falls off
        if cb>params['fil_length']:
            rx = Z[0,rfil];
            ry = Z[1,rfil];
            rfil = -1;
        # motor continues to travel along filament
        else:
            s = np.sqrt((Z[0,rfil] - rx)**2 + (Z[1,rfil] - ry)**2) - (params['v']*params['tstep']);
            rx = Z[0,rfil] - (s*np.cos(Z[2,rfil]));
            ry = Z[1,rfil] - (s*np.sin(Z[2,rfil]));
        # update location of bundled motor
        lx = rx;
        ly = ry;
        
    # right motor not bound, left motor bound to filament; left motor walks    
    elif lfil!=-1 and rfil==-1:
        cb = np.sqrt((Z[3,lfil] - lx)**2 + (Z[4,lfil] - ly)**2);
        # motor travels past end of filament, falls off
        if cb>params['fil_length']:
            lx = Z[0,lfil];
            ly = Z[1,lfil];
            lfil = -1;
        # motor continues to travel along filament
        else:
            s = np.sqrt((Z[0,lfil] - lx)**2 + (Z[1,lfil] - ly)**2) - (params['v']*params['tstep']);
            lx = Z[0,lfil] - (s*np.cos(Z[2,lfil]));
            ly = Z[1,lfil] - (s*np.sin(Z[2,lfil]));
        # update location of bundled motor
        rx = lx;
        ry = ly;
        
    # both motors bound to separate filaments
    elif lfil!=-1 and rfil!=-1 and lfil!=rfil:
        # calculate travel distance for left motor
        cb = np.sqrt((Z[3,lfil] - lx)**2 + (Z[4,lfil] - ly)**2);
        # motor travels past end of filament, falls off
        if cb>params['fil_length']:
            lx = Z[0,lfil];
            ly = Z[1,lfil];
            lfil = -1;
        # motor continues to travel along filament
        else:
            s = np.sqrt((Z[0,lfil] - lx)**2 + (Z[1,lfil] - ly)**2) - (params['v']*params['tstep']);
            lx = Z[0,lfil] - (s*np.cos(Z[2,lfil]));
            ly = Z[1,lfil] - (s*np.sin(Z[2,lfil]));
        # calculate travel distance for right motor
        cb = np.sqrt((Z[3,rfil] - rx)**2 + (Z[4,rfil] - ry)**2);
        # motor travels past end of filament, falls off
        if cb>params['fil_length']:
            rx = Z[0,rfil];
            ry = Z[1,rfil];
            rfil = -1;
        # motor continues to travel along filament
        else:
            s = np.sqrt((Z[0,rfil] - rx)**2 + (Z[1,rfil] - ry)**2) - (params['v']*params['tstep']);
            rx = Z[0,rfil] - (s*np.cos(Z[2,rfil]));
            ry = Z[1,rfil] - (s*np.sin(Z[2,rfil]));      
        # calculate new distance between bundled motors
        motor_stretch = np.sqrt((lx-rx)**2 + (ly-ry)**2);
        # check if bundled motors have passed stretch threshold
        # if so, both motors "fall off" and unbind
        if motor_stretch>params['r']:
            lfil = -1;
            rfil = -1;
            # equal chance of one motor moving to position of other motor
            if np.random.uniform()>0.5:
                lx = rx;
                ly = ry;
            else:
                rx = lx;
                ry = ly;
                
    # both motors bound to the same filament    
    elif lfil==rfil and lfil!=-1:
        # calculate travel distance for left motor
        cb = np.sqrt((Z[3,lfil] - lx)**2 + (Z[4,lfil] - ly)**2);    
        # motor travels past end of filament, falls off
        if cb>params['fil_length']:
            lx = Z[0,lfil];
            ly = Z[1,lfil];
            lfil = -1;
            rfil = -1;
        # motor continues to travel along filament
        else:
            s = np.sqrt((Z[0,lfil] - lx)**2 + (Z[1,lfil] - ly)**2) - (params['v']*params['tstep']);
            lx = Z[0,lfil] - (s*np.cos(Z[2,lfil]));
            ly = Z[1,lfil] - (s*np.sin(Z[2,lfil]));
        # update location of bundled motor
        rx = lx;
        ry = ly;

    return lfil,rfil,lx,ly,rx,ry        

def depolymerization(J,X,fil,params,boundary):
    # remove all bound motors from filament
    for j in range(0,params['motorsX']):
        # check if motor is active and bundled
        if J[0,j] == 2:
            # if motor j is on filament, but motor bundled to j is not
            if J[2,j]==fil and J[2,J[1,j]]!=fil:
                # mark motor j as unbound and move it to position of bundled motor
                J[2,j] = -1;
                X[0,j] = X[0,J[1,j]];
                X[1,j] = X[1,J[1,j]];
            # if motor bundled to j is on filament, but motor j is not
            elif J[2,j]!=fil and J[2,J[1,j]]==fil:
                # mark bundled motor as unbound and move it to position of motor j
                J[2,J[1,j]] = -1;
                X[0,J[1,j]] = X[0,j];
                X[1,J[1,j]] = X[1,j];
            # if both bundled motors are on filament
            elif J[2,j]==fil and J[2,J[1,j]]==fil:
                # mark both bundled motors as unbound
                J[2,j] = -1;
                J[2,J[1,j]] = -1;
                # equal probability of one motor to move to position of the other
                if np.random.uniform()>0.5:
                    X[0,j] = X[0,J[1,j]];
                    X[1,j] = X[1,J[1,j]];
                else:
                    X[0,J[1,j]] = X[0,j];
                    X[1,J[1,j]] = X[1,j];               
    # reset position of filament (to keep number of filaments constant
    # by assuming for each filament deploymerized, one is polymerized)
    x_plus_end = 2*params['boundary_width']*np.random.uniform()-params['boundary_width'];
    y_plus_end = 2*params['boundary_height']*np.random.uniform()-params['boundary_height'];
    while 1:
        angle = 2*np.pi*np.random.uniform();
        x_minus_end = x_plus_end-params['fil_length']*np.cos(angle);
        y_minus_end = y_plus_end-params['fil_length']*np.sin(angle);
        # check if minus ends of filament are within boundary
        if boundary.contains_point(np.array([x_minus_end,y_minus_end])):
            break 
    fil_params = [x_plus_end,y_plus_end,angle,x_minus_end,y_minus_end];

    return J,X,x_plus_end,y_plus_end,angle,x_minus_end,y_minus_end,0
def transplant_plus(x,y,ctr,rad):
    if not (ctr[0]==x and ctr[1]==y): # point not at center
        if x == ctr[0]: # infinite slope
            if y>ctr[1]:
                xnew = x
                ynew = rad
            elif y<ctr[1]:
                xnew = x
                ynew = -rad
        elif y == ctr[1]: # zero slope
            if x>ctr[0]:
                xnew = rad
                ynew = y
            elif x<ctr[0]:
                xnew = -rad
                ynew = y
        else: # not infinite or zero slope
            y_dist = (y-ctr[1])
            x_dist = (x-ctr[0])
            slope = y_dist/x_dist
            theta = np.arctan(slope)
            if x_dist>0 and y_dist>0: # point in quadrant I
                xnew = rad * np.cos(theta)
                ynew = rad * np.sin(theta)
            elif x_dist<0 and y_dist>0: # point in quadrant II
                xnew = -rad * np.cos(theta)
                ynew = -rad * np.sin(theta)
            elif x_dist<0 and y_dist<0: # point in quadrant III
                xnew = -rad * np.cos(theta)
                ynew = -rad * np.sin(theta)            
            elif x_dist>0 and y_dist<0: # point in quadrant IV
                xnew = rad * np.cos(theta)
                ynew = rad * np.sin(theta)
        return xnew+ctr[0],ynew+ctr[1]
    else:
        # point at center of circle, choose random point on circumference
        theta = np.random.rand()*2*np.pi
        xnew = rad*np.cos(theta)
        ynew = rad*np.sin(theta)

def force_on_filament(J,X,xp,yp,angle,igg_bound,fil,params):
    # frictional coefficients - based on Hunt, Gittes, Howard
    eta = 0.6;  # pN*s/um^2  or  Pa*s
    di = 0.008; # diameter of actin filament in um
    length = params['fil_length'];

    p = length/di;

    gparl = -0.2;
    gperp = 0.84;
    grot = -0.662;  
    l_pr = (2*np.pi*eta*length)/(np.log(p) + gparl);
    l_pp = (4*np.pi*eta*length)/(np.log(p) + gperp);
    l_r  = ((1/3)*np.pi*eta*(length**3))/(np.log(p) + grot);
    # thermal diffusion of filament paramters
    KT = 0.005 * 10;
    Dpr = KT/l_pr;
    Dpp = KT/l_pp;
    Dr  = KT/l_r;

    # pre_allocate force vector component array and torque array
    force_vectors = np.zeros((2,params['motorsX']));
    torques = np.zeros(params['motorsX']);

    # rotation matrix and inverse
    rot = np.array([[np.cos(angle),np.sin(angle)],[-1*np.sin(angle),np.cos(angle)]]);
    rot_inv = np.array([[np.cos(angle),-1*np.sin(angle)],[np.sin(angle),np.cos(angle)]]);

    # center of mass position of filament
    COMx = (xp + (xp - length*np.cos(angle)))/2;
    COMy = (yp + (yp - length*np.sin(angle)))/2;
    COM = np.array([[COMx],[COMy]]);
    # center of mass position in rotated coordinate system; "matmul = matrix multiplication"
    COM_rot = np.matmul(rot,COM);


    # every motor on a filament (group1)
    v1 = np.where(J[2,:]==fil)[0];
    # bundle index for each motor in v1 (group2)
    v2 = J[1,v1].astype(int)
    # bound filament index for v2 motors (group3)
    v3 = J[2,v2];
    # motors of group4 that are not bound to a filament
    r1 = np.where(v3==-1)[0];
    # motor bundles where both legs are bound to same filament
    r2 = np.where(fil==v3)[0];
    # group1 with exclusions (r1 and r2)
    g1 = np.delete(v1,np.union1d(r1,r2));

    
    # position of first motor
    x1 = X[0,g1];
    y1 = X[1,g1];
    # position of second motor
    x2 = X[0,J[1,g1].astype(int)];
    y2 = X[1,J[1,g1].astype(int)];
    # compute x and y components of force vector
    F_x = params['k']*(x2 - x1);
    F_y = params['k']*(y2 - y1);


    # compute length of motor from center of mass of filament
    lever = np.sqrt((x1 - COMx)**2 + (y1 - COMy)**2);
    # check which side of COM the motor is
    for mot in range(0,len(lever)):
        if np.sqrt((xp - x1[mot])**2 + (yp - y1[mot])**2)>length/2:
            lever[mot] = -1*lever[mot];


    # rotate x and y components into parallel and perpendicular components
    # then insert parallel and perpindicular components of force vector into array
    force_vectors[0,g1] = rot[0,0]*F_x + rot[0,1]*F_y;
    force_vectors[1,g1] = rot[1,0]*F_x + rot[1,1]*F_y;
    torques[g1] = lever*force_vectors[1,g1];
    
    # pre-allocate array for new COM positions in rotated coordinate system      
    COM_rot_new = np.zeros([2,1]);    
    if igg_bound != 1:
        # new parallel component of COM of filament
        COM_rot_new[0] = COM_rot[0] + \
                         params['tstep']*np.sum(force_vectors[0,:])/l_pr + \
                         np.sqrt(2*params['tstep']*Dpr)*np.random.randn();
        # new perpendicular component of COM of filament
        COM_rot_new[1] = COM_rot[1] + \
                         params['tstep']*np.sum(force_vectors[1,:])/l_pp + \
                         np.sqrt(2*params['tstep']*Dpp)*np.random.randn();
        # rotate COM positions in rotated cooridnate system back to standard coordinate system
        COM_new = np.matmul(rot_inv,COM_rot_new);
        # new angular component of filament
        angle_new = angle + \
                    params['tstep']*np.sum(torques)/l_r + \
                    np.sqrt(2*params['tstep']*Dr)*np.random.randn();
        # update positions of plus end of filament
        xp_new = COM_new[0] + (length/2)*np.cos(angle_new) + (0.008 * np.random.randn() + 0.005);
        yp_new = COM_new[1] + (length/2)*np.sin(angle_new);
        
    else:
        ##**** HOW TO INCORPORATE LEVER PARAMTER FOR ACTIN W/ BOUND MOTOR AND ON IGG DOMAIN? ****##
        angle_new = angle + \
                    params['tstep']*np.sum(torques)/l_r + \
                    np.sqrt(2*params['tstep']*Dr)*np.random.randn();
        xp_new = xp
        yp_new = yp
    
    return xp_new,yp_new,angle_new
def filament_boundary_check(Z,f,params,boundary):
    xp = Z[0,f]; 
    yp = Z[1,f];
    angle = Z[2,f];
    xm = Z[3,f];
    ym = Z[4,f];
    
    h = params['boundary_height'];
    w = params['boundary_width'];
    # check if plus end of filament is within boundary
    if ~boundary.contains_point(np.array([xp,yp])):
        b = [np.less(yp,-1*h),np.greater(yp,h),np.less(xp,-1*w),np.greater(xp,w)];
        # b[0] = yp position below bottom boundary
        # b[1] = yp position above top boundary
        # b[2] = xp position left of left boundary
        # b[3] = xp position right of right boundary
        if b[0] == True and b[2] == False and b[3] == False:
            yp = -1*h;
            ym = yp - params['fil_length']*np.sin(angle);
        elif b[1] == True and b[2] == False and b[3] == False:
            yp = h;
            ym = yp - params['fil_length']*np.sin(angle);
        elif b[2] == True and b[0] == False and b[1] == False:
            xp = -1*w;
            xm = xp - params['fil_length']*np.cos(angle);
        elif b[3] == True and b[0] == False and b[1] == False:
            xp = w;
            xm = xp - params['fil_length']*np.cos(angle);
        elif b[0] == True and b[2] == True:
            xp = -1*w;
            yp = -1*h;
            xm = xp - params['fil_length']*np.cos(angle);
            ym = yp - params['fil_length']*np.sin(angle);
        elif b[0] == True and b[3] == True:
            xp = w;
            yp = -1*h;
            xm = xp - params['fil_length']*np.cos(angle);
            ym = yp - params['fil_length']*np.sin(angle);
        elif b[1] == True and b[2] == True:
            xp = -1*w;
            yp = h;
            xm = xp - params['fil_length']*np.cos(angle);
            ym = yp - params['fil_length']*np.sin(angle);
        elif b[1] == True and b[3] == True:
            xp = w;
            yp = h; 
            xm = xp - params['fil_length']*np.cos(angle);
            ym = yp - params['fil_length']*np.sin(angle);
    # check if minus end of filament is within boundary
    if ~boundary.contains_point(np.array([xm,ym])):
        b = [np.less(ym,-1*h),np.greater(ym,h),np.less(xm,-1*w),np.greater(xm,w)];
        # b[0] = ym position below bottom boundary
        # b[1] = ym position above top boundary
        # b[2] = xm position left of left boundary
        # b[3] = xm position right of right boundary
        if b[0] == True and b[2] == False and b[3] == False:
            ym = -1*h;
            yp = ym + params['fil_length']*np.sin(angle);
        elif b[1] == True and b[2] == False and b[3] == False:
            ym = h;
            yp = ym + params['fil_length']*np.sin(angle);
        elif b[2] == True and b[0] == False and b[1] == False:
            xm = -1*w;
            xp = xm + params['fil_length']*np.cos(angle);
        elif b[3] == True and b[0] == False and b[1] == False:
            xm = w;
            xp = xm + params['fil_length']*np.cos(angle);
        elif b[0] == True and b[2] == True:
            xm = -1*w;
            ym = -1*h;
            xp = xm + params['fil_length']*np.cos(angle);
            yp = ym + params['fil_length']*np.sin(angle);
        elif b[0] == True and b[3] == True:
            xm = w;
            ym = -1*h;
            xp = xm + params['fil_length']*np.cos(angle);
            yp = ym + params['fil_length']*np.sin(angle);
        elif b[1] == True and b[2] == True:
            xm = -1*w;
            ym = h;
            xp = xm + params['fil_length']*np.cos(angle);
            yp = ym + params['fil_length']*np.sin(angle);
        elif b[1] == True and b[3] == True:
            xm = w;
            ym = h; 
            xp = xm + params['fil_length']*np.cos(angle);
            yp = ym + params['fil_length']*np.sin(angle);

    return [xp,yp,angle,xm,ym]

In [ ]:
# INITIALIZATION - DEFINE PARAMETERS HERE 
from PIL import Image, ImageDraw
# define dictionary to store all parameters
params = {
# image parameters
'save_image':1, #0=No, 1=Yes
'res':1, #set resolution (r=1 == s[0] x s[1] pixels)
's':(600,300), # size of image in pixels
'image_iter':1,
'image_save_path':"./TEST_igg_images_tiffs/",
'tiff_save_name':"sim_tiff8.tiff",

# simulation parameters
'time':2000, #iterations of simulation
'tstep':0.005, #time step between each iteration
'filaments':2000, #number of filaments
'motorsX':3000, #number of myosin X motors
'motorsI':600, #number of myosin I motors

# filament parameters
'fil_length':0.2, #filament length
'p0':1.0, #detachment rate
'p1':0.2, #attachement rate
'p2':0.0, #depolymerization rate 
'depoly_rate':0, #depolymerization rate
'igg_attach_rate':0.9, #IgG binding rate for plus ends of filaments to IgG domain
'igg_detach_rate':0.0, #IgG detachment rate for plus ends of filaments from IgG domain
    
# myosinX motor parameters
'r':0.3, #radius of search for motor attachment & max stretch for walk
'v':1.0, #velocity of motor
'k':3.0, #stiffness of motor 'spring'
'diffusionRate':0.0029,
'unbundleRate':1.0,
'bundleRate':0.2,
'activationRate':3.0,
'inactivationRate':1.0,
'motorBundleRadius':0.3, # same as r according to literature
    
# myosinI motor parameters
'r_I':0.2, #radius of parameter to find filament once bound to IgG domain
'myoI_igg_on':1.0, #binding rate to IgG domain
'myoI_igg_off':0.0, #unbinding rate from IgG domain

# edge boundary parameters
'boundary_width':2.0,
'boundary_height':1.0,

# IgG domain parameters
'igg_radius':0.4,
'igg_center':[0.5,0.0],
    
# initial zoning parameter
'init_zone_factor':2.6
}

if params['save_image'] == 1:
    if not path.exists(params['image_save_path']):
        makedirs(params['image_save_path'])
    
params['s'] = np.asarray((params['s'][0]*params['res'],params['s'][1]*params['res']))

# pre-allocate memory for output arrays
A = np.zeros((4,params['motorsI'])); #A == 1st leter, myosinI
J = np.zeros((3,params['motorsX'])); #J == 10th letter, myosinX
J[[1,2],:] = -1;
X = np.zeros((2,params['motorsX']));
Z = np.zeros((6,params['filaments']));

# J -> myosinX motor state output:
#    J(0,:) = progression of motor state
#        0-inactive
#        1-active
#        2-bundled & active
#    J(1,:) = index of motor bundled to (-1 means no bundle)
#    J(2,:) = index of filament bound to (-1 means no binding to filament)
# X -> myosinX motor position output:
#    X(0,:) = x position of motor head
#    X(1,:) = y position of motor head
# A -> myosinI motor position output:
#    A(0,:) = x position of motor head
#    A(1,:) = y position of motor head
#    A(2,:) = filament bound to by myosinI motor
#    A(3,:) = indicator of binding to IgG domain
# Z -> filament output:
#    Z(0,:) = x position of filament +end
#    Z(1,:) = y position of filament +end
#    Z(2,:) = angle of orientation (unit circle, CCW)
#    Z(3,:) = Z(1,:) - (fil_length * cos(Z(3,:))) x position of filament -end
#    Z(4,:) = Z(2,:) - (fil_length * sin(Z(3,:))) y position of filament -end
#    Z(5,:) = indicator of binding to IgG domain

def initialize_positions(params,J,X,Z):
    boundary_height = params['boundary_height'];
    boundary_width = params['boundary_width'];
    boundary = pltpath.Path([(-1*boundary_width,-1*boundary_height),
                          (-1*boundary_width,boundary_height),
                          (boundary_width,boundary_height),
                          (boundary_width,-1*boundary_height)],
                          readonly=True)
    
    # assign random positions to ends of filaments
    for i in range(0,params['filaments']):
        # randomly assign positions to plus ends of filaments
        Z[0,i] = 2*(boundary_width/params['init_zone_factor'])*np.random.uniform()-boundary_width;
        Z[1,i] = 2*boundary_height*np.random.uniform()-boundary_height;
        while 1:
            angle = 2*np.pi*np.random.uniform();
            x_minus_end = Z[0,i]-params['fil_length']*np.cos(angle);
            y_minus_end = Z[1,i]-params['fil_length']*np.sin(angle);
            # check if minus ends of filament are within boundary
            if boundary.contains_point(np.array([x_minus_end,y_minus_end])):
                Z[2,i] = angle;
                Z[3,i] = x_minus_end;
                Z[4,i] = y_minus_end;
                break      
            
    # assign random positions to motors
    for j in range(0,params['motorsX']):
            X[0,j] = 2*(boundary_width/params['init_zone_factor'])*np.random.uniform()-boundary_width;
            X[1,j] = 2*boundary_height*np.random.uniform()-boundary_height;         
            
    return boundary,J.astype(int),X,Z

 
# initialize positions
boundary,J,X,Z = initialize_positions(params,J,X,Z);

# write image data to file
if params['save_image']==1:
    im_png = Image.fromarray(image_build(J,X,Z,params))
    im_png.save(params['image_save_path']+'sim_test0.png')
    imshow(im_png)
    
    im_tiff = image_build(J,X,Z,params)
    imsave(params['image_save_path']+params['tiff_save_name'],im_tiff,compress=1) # for TIFF file
    # imshow(im_tiff)

In [ ]:
# SIMULATION HEAD
from PIL import Image, ImageDraw
from tqdm import tqdm

def sim(J,X,Z,params,boundary):
    
    for t in tqdm(range(0,params['time'])):
        #determine motor state
        #motor state forward 0->1->2
        #motor state backward 2->1->0  
        tempJ = J;
        # apply forward change of states on motors
        for j in range(0,params['motorsX']):
            if tempJ[0,j] != 2 :
                tempJ[0,j],tempJ[1,j],X = motor_state_forward(j,tempJ,X,params,boundary);
                if tempJ[0,j] == 2 and tempJ[0,tempJ[1,j]] != 2:
                    tempJ[0,tempJ[1,j]] = 2;
                    tempJ[1,tempJ[1,j]] = j;
                   
            if tempJ[0,j] == 2 and J[0,j] != 2 and tempJ[2,j] == -1 and tempJ[2,tempJ[1,j]] == -1 and \
               j<tempJ[1,j]:
                    X[0,j] = (X[0,j] + X[0,tempJ[1,j]])/2;
                    X[1,j] = (X[1,j] + X[1,tempJ[1,j]])/2;
                    X[0,tempJ[1,j]] = X[0,j];
                    X[1,tempJ[1,j]] = X[1,j];
        
        # apply backward change of states on motors
        for j in range(0,params['motorsX']):
            if tempJ[0,j] == 1 or (tempJ[0,j] == 2 and tempJ[2,j] == -1 and tempJ[2,tempJ[1,j]] == -1):
                if tempJ[0,j] == 1 and params['inactivationRate']*params['tstep']>np.random.uniform():
                    tempJ[0,j] = 0;
                elif tempJ[0,j] == 2 and params['unbundleRate']*params['tstep']>np.random.uniform():
                    tempJ[0,tempJ[1,j]] = 1;
                    tempJ[1,tempJ[1,j]] = -1;
                    tempJ[0,j] = 1;
                    tempJ[1,j] = -1;
                    
        J = tempJ.astype(int);

        
        # check if motors attach or detach from filaments
        # call sub-routine if motor is bundled and has not been edited yet
        cands = np.where(J[1,:]>J[1,J[1,:]])[0];
        if len(cands>0):
            for j in cands:
                J[2,j],J[2,J[1,j]],X[0,j],X[1,j],X[0,J[1,j]],X[1,J[1,j]] = \
                motor_attach_detach(J[2,j],J[2,J[1,j]],
                                    X[0,j],X[1,j],
                                    X[0,J[1,j]],X[1,J[1,j]],
                                    Z,params,boundary);

                
        # determine movement of motors that are bound to filaments
        cands = np.where( ((J[2,:]!=-1) | (J[2,J[1,:]]!=-1)) & (J[1,:]>J[1,J[1,:]]) )[0];
        # for j in range(0,params['motors']):
        for j in cands:
            # call sub-routine if motor is bundled, attached to a filament and has not been edited yet
            J[2,j],J[2,J[1,j]],X[0,j],X[1,j],X[0,J[1,j]],X[1,J[1,j]] = \
            bound_motor_movement(J[2,j],J[2,J[1,j]],
                                 X[0,j],X[1,j],
                                 X[0,J[1,j]],X[1,J[1,j]],
                                 Z,params);
               
 
        # check if any filaments undergo depolymerization
        for f in range(0,params['filaments']):
            # check if filament depolymerizes
            if params['p2']*params['tstep']>np.random.uniform():
                J,X,Z[0,f],Z[1,f],Z[2,f],Z[3,f],Z[4,f],Z[5,f] = depolymerization(J,X,f,params,boundary);
                
        
        # # find myosinI motors that are bound to IgG domain
        # myoI_cands = np.where(A(3,:)==1)[0]
        # for a in myoI:
        #     # if myoI bound to IgG domain currently, find a filament to bind to
        #     if a in myoI_cands and A[2,a]==-1:
        #         # find filaments that are bound to IgG domain
        #         cand_filsI = np.where( (Z[5,:]==1) & (np.sqrt((Z[0,:]-A[0,a])**2 + (Z[1,:]-A[1,a])**2)<params['r_I']) )[0]
        #         if len(cand_filsI)>0:
        #             A[2,a] = cand_filsI[int(np.ceil(len(cand_filsI)*np.random.uniform()))-1]
        #     # check if myoI motor binds to IgG domain
        #     elif np.sqrt((A[0,a]-params['igg_center'][0])**2 + (A[1,a]-params['igg_center'][1])**2)<params['igg_radius']:
        #         A[0,a],A[1,a] = transplant_plus(A[0,a],A[1,a],np.asarray(params['igg_center']),params['igg_radius'])
        #         A[3,a] = 1
        #     # myoI motor diffuses (same diffusion rate as myoX motors; change?)
        #     else:
        #         A[0,a] = np.sqrt(2*params['tstep']*params['diffusionRate'])*np.random.randn();
        #         A[1,a] = np.sqrt(2*params['tstep']*params['diffusionRate'])*np.random.randn();
                
        
        # check if filament plus ends attach to IgG domain
        # find all filaments where plus end of filament is within IgG domain
        cands = np.where(np.sqrt((Z[0,:]-params['igg_center'][0])**2 + (Z[1,:]-params['igg_center'][1])**2)<params['igg_radius'])[0]
        for f in cands:
            # lock plus end of filament to IgG domain
            if Z[5,f] == 0 and params['igg_attach_rate']>np.random.uniform():
                Z[5,f] = 1
                # # attach plus end of filament to closest point on edge of IgG domain
                # Z[0,f],Z[1,f] = transplant_plus(Z[0,f],Z[1,f],np.asarray(params['igg_center']),params['igg_radius'])
                # # update minus end of filament, retain angle orientation   
                # Z[3,f] = Z[0,f] - params['fil_length']*np.cos(Z[2,f]);
                # Z[4,f] = Z[1,f] - params['fil_length']*np.sin(Z[2,f]);   
            # unlock plus end of filament from IgG domain       
            elif Z[5,f] == 1 and params['igg_detach_rate']>np.random.uniform():
                Z[5,f] = 0
        
        
        # iteratively determine filament movement
        for f in range(0,params['filaments']):
            # determine force on filament
            Z[0,f],Z[1,f],Z[2,f] = force_on_filament(J,X,Z[0,f],Z[1,f],Z[2,f],Z[5,f],f,params);
            Z[3,f] = Z[0,f] - params['fil_length']*np.cos(Z[2,f]);
            Z[4,f] = Z[1,f] - params['fil_length']*np.sin(Z[2,f]);
            # check if filament hit boundaries
            Z[:5,f] = filament_boundary_check(Z[:5],f,params,boundary);
            
            # move motors attached to filament accordingly
            # find motors on filament
            mots1 = np.where((J[0,:]==2) & (J[0,J[1,:]]==2) & (J[2,:]!=J[2,J[1,:]]) & (J[2,:]!=-1) & (J[2,J[1,:]]!=-1) & (J[2,:]==f))[0];
            # distance between motor and plus end of filament
            s = np.sqrt((Z[0,f] - X[0,mots1])**2 + (Z[1,f] - X[1,mots1])**2);
            # x and y movement of motor to be on filament
            X[0,mots1] = Z[0,f] - s*np.cos(Z[2,f]);
            X[1,mots1] = Z[1,f] - s*np.sin(Z[2,f]);

            # find motors with bundled motor on filament
            mots2 = np.where((J[0,:]==2) & (J[0,J[1,:]]==2) & (J[2,:]!=J[2,J[1,:]]) & (J[2,:]!=-1) & (J[2,J[1,:]]!=-1) & (J[2,J[1,:]]==f))[0];
            # distance between motor and plus end of filament
            s = np.sqrt((Z[0,f] - X[0,J[1,mots2]])**2 + (Z[1,f] - X[1,J[1,mots2]])**2);
            # x and y movement of motor to be on filament
            X[0,J[1,mots2]] = Z[0,f] - s*np.cos(Z[2,f]);
            X[1,J[1,mots2]] = Z[1,f] - s*np.sin(Z[2,f]);  
    
        
        # check if any bundled motors have surpassed radius limit
        # every motor on a filament (group1)
        v1 = np.where(J[2,:]!=-1)[0];
        # bundle index for each motor in v1 (group2)
        v2 = J[1,v1].astype(int);
        # bound filament index for v1 motors (group3)
        v3 = J[2,v1];
        # bound filament index for v2 motors (group4)
        v4 = J[2,v2];
        # motors of group4 that are not bound to a filament
        r1 = np.where(v4==-1)[0];
        # motor bundles where both legs are bound to same filament
        r2 = np.where(v3==v4)[0];
        # group1 with exclusions (r1 and r2)
        g1 = np.delete(v1,np.union1d(r1,r2));
        # find stretch distances between candidate motors
        motor_stretch = np.sqrt((X[0,g1] - X[0,J[1,g1]])**2 + (X[1,g1] - X[1,J[1,g1]])**2);    
        # find indices of stretch that surpass bundled motor radius
        bar = np.where(motor_stretch>params['motorBundleRadius'])[0];
        
        # for bundled pairs passing radius, randomly select a leg to unbind from filament and move to other leg's position
        for fo in bar:
            foo = g1[fo]
            if np.random.uniform()<0.5:
                J[2,foo] = -1;
                X[0,foo] = X[0,J[1,foo]];
                X[1,foo] = X[1,J[1,foo]];
            else:
                J[2,J[1,foo]] = -1;
                X[0,J[1,foo]] = X[0,foo];
                X[1,J[1,foo]] = X[1,foo];  
                
        
        # write image data to '.tiff' file
        if params['save_image']==1:
            if (t+params['image_iter']+1)%params['image_iter'] == 0:
                time_point = str(int(((t+1)/params['image_iter'])))
                im_png = Image.fromarray(image_build(J,X,Z,params))
                im_png.save(params['image_save_path']+'sim_test'+time_point+'.png')
                
                im_tiff = image_build(J,X,Z,params)
                imsave(params['image_save_path']+params['tiff_save_name'],im_tiff,append=True,compress=1) # for TIFF file
    
    # finish and pass data back
    return J,X,Z

In [ ]:
# RUN THE SIMULATION 

J,X,Z = sim(J,X,Z,params,boundary)

# import cProfile
# cProfile.run('sim2(J,X,Z,params,boundary)')

In [ ]:
# VIEW IMAGES
import glob
from IPython.html.widgets import interact
from IPython.display import Image, display

cache = {}
def get_image(key):
    if not key in cache:
        cache[key] = open(key,'rb').read()
    return cache[key]

images = glob.glob(params['image_save_path']+'*.png')
for image in images:
    get_image(image)

def pltimg(Time=0):
    filename = params['image_save_path']+'sim_test'+str(Time)+'.png'
    display(Image(data=cache[filename]))

interact(pltimg, Time=(0,params['time'],params['image_iter']))

In [ ]:
# CONVERT '.PNG' IMAGES INTO A '.TIFF' STACK
tiff_name = 'sim_tiffx.tiff'
path = params['image_save_path']

from scipy import misc
first = 0
for image_path in tqdm(glob.glob(params['image_save_path']+'*.png')):
    im = misc.imread(image_path)
    if first == 0:
        imsave(path+tiff_name,im,compress=1)
        first = 1
    else:
        imsave(path+tiff_name,im,append=True,compress=1)